# Downloading Dataset to Kaggle

In [ ]:

url = 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/meta_categories/meta_Cell_Phones_and_Accessories.jsonl.gz'
save_path = 'meta.jsonl.gz'  
response = requests.get(url)
if response.status_code == 200:
    with open(save_path, 'wb') as file:
        file.write(response.content)
    print(f"Dataset downloaded successfully to {save_path}")
else:
    print(f"Failed to download dataset, status code {response.status_code}")

In [ ]:

# Specify the path to your JSONL.gz file
file_path = 'meta.jsonl.gz'
target_asin = asin_l

def records_with_asin(file_path, target_asin):
    matched_records = []
    with gzip.open(file_path, 'rt') as file:
        for line in file:
            record = json.loads(line)
            if record.get('parent_asin') in target_asin:
                matched_records.append(record)
    return matched_records

# Example usage to convert matched records to DataFrame
def count_records_with_asin(file_path, target_asin):
    try:
        matched_records = records_with_asin(file_path, target_asin)
        df_matched = pd.DataFrame(matched_records)
        return df_matched
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        
try:
    df_matched = count_records_with_asin(file_path, target_asin)
    if df_matched is not None:
        print("Matched records:")
        print(df_matched)
except Exception as e:
    print(f"An error occurred: {str(e)}")


# Selecting Asin for Required product in Product Data

In [ ]:
import re
pattern = re.compile(r'Apple iPhone \d+ \w+,? \d+GB,? [\w\s]+.*', re.IGNORECASE)
#pattern2 = re.compile(r'Samsung Galaxy [\w]*,? \w* ,? \d+GB \w*,?.*', re.IGNORECASE   )

# Search for the pattern in the 'Product_Description' column
matches = df_meta['title'].apply(lambda x: bool(pattern.match(x)))

# Filter the DataFrame based on the matches
filtered_df = df_meta[matches]

print(filtered_df.shape)
apple1 = filtered_df['parent_asin'].tolist()
print(apple1)

In [ ]:
import gzip
import json

def read_jsonl_gz(file_path):
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            yield json.loads(line)

# Create DataFrames from generators

df_meta = pd.DataFrame(read_jsonl_gz('meta.jsonl.gz'))

# Filtering Products Based on Keywords

In [ ]:
keywords = ['Apple', 'iPhone','Samsung']

filtered_df = df_meta[~df_meta['title'].str.contains('|'.join(keywords), case=False, na=False)]
#filtered_df = filtered_df.dropna()

print(filtered_df)

In [ ]:
filtered_df = filtered_df.drop(columns=['videos','categories','bought_together','subtitle', 'author','main_category'])
filtered_df.shape

In [ ]:
keywords = ['Cell Phones & Accessories']

filtered_df = df_meta[df_meta['main_category'].str.contains('|'.join(keywords), case=False, na=False)]
#filtered_df = filtered_df.dropna()

print(filtered_df)

In [ ]:
keywords = ['pixel','Pixel','google','Google']

filtered_df_na = filtered_df_na[filtered_df_na['title'].str.contains('|'.join(keywords), case=False, na=False)]
#filtered_df = filtered_df.dropna()

print(filtered_df_na)

In [ ]:
filtered_df_na = filtered_df.dropna(subset=['price'])
filtered_df_na = filtered_df_na[filtered_df_na['price'] > 60]
print(filtered_df_na.head)
print(filtered_df_na.shape)

In [ ]:
keywords_to_drop = ['Screen','iPod','Connector','Bag','Cover','Holder','Samsung','samsung','Apple','apple','iPhone','Galaxy']

# Filter out rows where 'Text' column contains any of the keywords
filtered_df = filtered_df[~filtered_df['title'].str.contains('|'.join(keywords_to_drop), case=False, na=False)]

filtered_df_na = filtered_df.dropna()
print(filtered_df)
print(filtered_df.shape)

In [ ]:
text_list = filtered_df_na['parent_asin'].tolist()
len(text_list)

In [ ]:
set1 = set(text_list)
set2 = set(apple1)

# Combine sets using the union operator "|"
combined_unique = list(set1 | set2)
len(combined_unique)

# Downloading Reviews Dataset

In [ ]:
import requests

# Replace 'your_dataset_url' with the actual URL of your dataset
url = 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/Cell_Phones_and_Accessories.jsonl.gz'
save_path = 'Cellphones.jsonl.gz'  # Specify the filename and extension based on your dataset

response = requests.get(url)
if response.status_code == 200:
    with open(save_path, 'wb') as file:
        file.write(response.content)
    print(f"Dataset downloaded successfully to {save_path}")
else:
    print(f"Failed to download dataset, status code {response.status_code}")


In [ ]:
import gzip
import json
from collections import Counter


def top_recurred_items(file_path, column_name, top_n=500):
    counts = Counter()
    
    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        for line in file:
            record = json.loads(line)
            if column_name in record:
                counts[record[column_name]] += 1

    top_items = counts.most_common(top_n)
    return top_items

# Example usage
file_path =  'Cellphones.jsonl.gz'
column_name = 'asin'  

top_items = top_recurred_items(file_path, column_name)
#print("Top recurring items:", top_items)

# Selecting Reviews Data Based on Asin

***Json File is Processed in Row wise due to large size***

In [ ]:
import gzip
import json
from collections import Counter
# Specify the path to your JSONL.gz file
file_path = 'Cellphones.jsonl.gz'
#target_asin = ['B0BY5V82W4', 'B0BXKWGCSW', 'B09JFJ1Q5C', 'B097MBC1P8', 'B08PNTRC2Y', 'B09KW9M48M', 'B0BY5V8P24', 'B099RCQZ8Z', 'B0BV7HZH2F', 'B08PNP57DQ', 'B09JFSMFB5', 'B0BY62TGRN', 'B0BZ1P8CXC', 'B09JFLKR3H', 'B09JFC967X', 'B09JFP3R1D', 'B0BJ1VRNSW', 'B09JF5ZHQS', 'B0BY5V81ST', 'B0BYKZ8TPF', 'B09JFQ9G5Z', 'B09JFQ3P7B', 'B09JFN8K6T', 'B09JFMBNPH', 'B09JF7QNZV', 'B09JFS4P67', 'B09JFTPQY1', 'B0BTMGQ7HY', 'B09JF3PVM2', 'B09JFSH31K', 'B09JFP32Y5', 'B0BV7J3KGX', 'B0BZ9N1QQC', 'B0CJ5PK4HF', 'B0BZZTWXNB', 'B0BWBCC8SV', 'B085T12N5L', 'B0BWCCVW35', 'B0BSLD9D79', 'B0CHZ84Z44', 'B0BQXGVMSL', 'B0C7D4JS8B', 'B0BZ9CCX5K', 'B0BZ9LJPLQ', 'B0BZ9NH985', 'B0CHZ8S5GY', 'B0C3WQW9KC']
target_asin = text_list
def records_with_asin(file_path, target_asin):
    matched_records = []
    with gzip.open(file_path, 'rt') as file:
        for line in file:
            record = json.loads(line)
            if record.get('asin') in target_asin:
                matched_records.append(record)
    return matched_records

# Example usage to convert matched records to DataFrame
def count_records_with_asin(file_path, target_asin):
    try:
        matched_records = records_with_asin(file_path, target_asin)
        df_matched = pd.DataFrame(matched_records)
        return df_matched
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        
try:
    df_matched = count_records_with_asin(file_path, target_asin)
    if df_matched is not None:
        print("Matched records:")
        print(df_matched)
except Exception as e:
    print(f"An error occurred: {str(e)}")


In [ ]:
df_matched.to_csv('pixel_products.csv', index=False)